In [1]:
import pandas as pd
import numpy as np
import json
import requests


In [2]:
df1 = pd.read_csv(r"C:/Users/HP/Desktop/Data Scientist/project/singapore resale flat price prediction/raw data/REF_19901999.csv")

In [3]:
df2 = pd.read_csv(r"C:/Users/HP/Desktop/Data Scientist/project/singapore resale flat price prediction/raw data/RFP_2000Feb2012.csv")

In [4]:
df3 = pd.read_csv(r"C:/Users/HP/Desktop/Data Scientist/project/singapore resale flat price prediction/raw data/REF_Mar2012toDec2014.csv")

In [5]:
df4 = pd.read_csv(r"C:/Users/HP/Desktop/Data Scientist/project/singapore resale flat price prediction/raw data/REF_Jan2015toDec2016.csv")

In [6]:
df5 = pd.read_csv(r"C:/Users/HP/Desktop/Data Scientist/project/singapore resale flat price prediction/raw data/REF_Jan2017onwards.csv")

In [7]:
df = pd.concat([df1,df2,df3,df4,df5],axis=0)

In [8]:
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,NaN
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,NaN
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,NaN
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,NaN
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
182350,2024-06,YISHUN,EXECUTIVE,328,YISHUN RING RD,04 TO 06,142.0,Apartment,1988,850000.0,63 years 01 month
182351,2024-06,YISHUN,EXECUTIVE,723,YISHUN ST 71,10 TO 12,146.0,Maisonette,1986,790000.0,61 years
182352,2024-06,YISHUN,EXECUTIVE,826,YISHUN ST 81,10 TO 12,146.0,Maisonette,1988,1000000.0,62 years 08 months
182353,2024-06,YISHUN,EXECUTIVE,826,YISHUN ST 81,04 TO 06,146.0,Maisonette,1988,1000000.0,62 years 08 months


In [9]:
# Identify unique val in each col

for i in df.columns:
    print(i,":",df[i].nunique())

month : 414
town : 27
flat_type : 8
block : 2712
street_name : 584
storey_range : 25
floor_area_sqm : 212
flat_model : 34
lease_commence_date : 55
resale_price : 9537
remaining_lease : 726


In [10]:
df.isna().sum()

month                       0
town                        0
flat_type                   0
block                       0
street_name                 0
storey_range                0
floor_area_sqm              0
flat_model                  0
lease_commence_date         0
resale_price                0
remaining_lease        709050
dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 928558 entries, 0 to 182354
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                928558 non-null  object 
 1   town                 928558 non-null  object 
 2   flat_type            928558 non-null  object 
 3   block                928558 non-null  object 
 4   street_name          928558 non-null  object 
 5   storey_range         928558 non-null  object 
 6   floor_area_sqm       928558 non-null  float64
 7   flat_model           928558 non-null  object 
 8   lease_commence_date  928558 non-null  int64  
 9   resale_price         928558 non-null  float64
 10  remaining_lease      219508 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 85.0+ MB


In [12]:
# droping remaining_lease missing values
df = df.dropna()
df.isnull().sum()

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
resale_price           0
remaining_lease        0
dtype: int64

# MRT (Mass Rapid Transit System) Railway Transportation
*   Getting MRT coordinates for each city
*   so that we can calculate the distance afterwards using MRT coordinates


In [13]:
df['town'].unique()

array(['ANG MO KIO', 'BEDOK', 'BISHAN', 'BUKIT BATOK', 'BUKIT MERAH',
       'BUKIT PANJANG', 'BUKIT TIMAH', 'CENTRAL AREA', 'CHOA CHU KANG',
       'CLEMENTI', 'GEYLANG', 'HOUGANG', 'JURONG EAST', 'JURONG WEST',
       'KALLANG/WHAMPOA', 'MARINE PARADE', 'PASIR RIS', 'PUNGGOL',
       'QUEENSTOWN', 'SEMBAWANG', 'SENGKANG', 'SERANGOON', 'TAMPINES',
       'TOA PAYOH', 'WOODLANDS', 'YISHUN'], dtype=object)

In [14]:
list_of_mrt = [
    'Jurong East MRT Station',
    'Bukit Batok MRT Station',
    'Bukit Gombak MRT Station',
    'Choa Chu Kang MRT Station',
    'Yew Tee MRT Station',
    'Kranji MRT Station',
    'Marsiling MRT Station',
    'Woodlands MRT Station',
    'Admiralty MRT Station',
    'Sembawang MRT Station',
    'Canberra MRT Station',
    'Yishun MRT Station',
    'Khatib MRT Station',
    'Yio Chu Kang MRT Station',
    'Ang Mo Kio MRT Station',
    'Bishan MRT Station',
    'Braddell MRT Station',
    'Toa Payoh MRT Station',
    'Novena MRT Station',
    'Newton MRT Station',
    'Orchard MRT Station',
    'Somerset MRT Station',
    'Dhoby Ghaut MRT Station',
    'City Hall MRT Station',
    'Raffles Place MRT Station',
    'Marina Bay MRT Station',
    'Marina South Pier MRT Station',
    'Pasir Ris MRT Station',
    'Tampines MRT Station',
    'Simei MRT Station',
    'Tanah Merah MRT Station',
    'Bedok MRT Station',
    'Kembangan MRT Station',
    'Eunos MRT Station',
    'Paya Lebar MRT Station',
    'Aljunied MRT Station',
    'Kallang MRT Station',
    'Lavender MRT Station',
    'Bugis MRT Station',
    'Tanjong Pagar MRT Station',
    'Outram Park MRT Station',
    'Tiong Bahru MRT Station',
    'Redhill MRT Station',
    'Queenstown MRT Station',
    'Commonwealth MRT Station',
    'Buona Vista MRT Station',
    'Dover MRT Station',
    'Clementi MRT Station',
    'Chinese Garden MRT Station',
    'Lakeside MRT Station',
    'Boon Lay MRT Station',
    'Pioneer MRT Station',
    'Joo Koon MRT Station',
    'Gul Circle MRT Station',
    'Tuas Crescent MRT Station',
    'Tuas West Road MRT Station',
    'Tuas Link MRT Station',
    'Expo MRT Station',
    'Changi Airport MRT Station',
    'HarbourFront MRT Station',
    'Chinatown MRT Station',
    'Clarke Quay MRT Station',
    'Little India MRT Station',
    'Farrer Park MRT Station',
    'Boon Keng MRT Station',
    'Potong Pasir MRT Station',
    'Woodleigh MRT Station',
    'Serangoon MRT Station',
    'Kovan MRT Station',
    'Hougang MRT Station',
    'Buangkok MRT Station',
    'Sengkang MRT Station',
    'Punggol MRT Station',
    'Bras Basah MRT Station',
    'Esplanade MRT Station',
    'Promenade MRT Station',
    'Nicoll Highway MRT Station',
    'Stadium MRT Station',
    'Mountbatten MRT Station',
    'Dakota MRT Station',
    'MacPherson MRT Station',
    'Tai Seng MRT Station',
    'Bartley MRT Station',
    'Lorong Chuan MRT Station',
    'Marymount MRT Station',
    'Caldecott MRT Station',
    'Botanic Gardens MRT Station',
    'Farrer Road MRT Station',
    'Holland Village MRT Station',
    'one-north MRT Station',
    'Kent Ridge MRT Station',
    'Haw Par Villa MRT Station',
    'Pasir Panjang MRT Station',
    'Labrador Park MRT Station',
    'Telok Blangah MRT Station',
    'Bayfront MRT Station',
    'Bukit Panjang MRT Station',
    'Cashew MRT Station',
    'Hillview MRT Station',
    'Beauty World MRT Station',
    'King Albert Park MRT Station',
    'Sixth Avenue MRT Station',
    'Tan Kah Kee MRT Station',
    'Stevens MRT Station',
    'Rochor MRT Station',
    'Downtown MRT Station',
    'Telok Ayer MRT Station',
    'Fort Canning MRT Station',
    'Bencoolen MRT Station',
    'Jalan Besar MRT Station',
    'Bendemeer MRT Station',
    'Geylang Bahru MRT Station',
    'Mattar MRT Station',
    'Ubi MRT Station',
    'Kaki Bukit MRT Station',
    'Bedok North MRT Station',
    'Bedok Reservoir MRT Station',
    'Tampines West MRT Station',
    'Tampines East MRT Station',
    'Upper Changi MRT Station',
    'Woodlands North MRT Station',
    'Woodlands South MRT Station'
]

In [15]:
mrt_lat = []
mrt_long = []

for i in range(0, len(list_of_mrt)):
    query_address = list_of_mrt[i]
    query_string = 'https://www.onemap.gov.sg/api/common/elastic/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
    resp = requests.get(query_string)

    data_mrt=json.loads(resp.content)

    if data_mrt['found'] != 0:
        mrt_lat.append(data_mrt["results"][0]["LATITUDE"])
        mrt_long.append(data_mrt["results"][0]["LONGITUDE"])

        print (str(query_address)+",Lat: "+data_mrt['results'][0]['LATITUDE'] +" Long: "+data_mrt['results'][0]['LONGITUDE'])

    else:
        mrt_lat.append('NotFound')
        mrt_lat.append('NotFound')
        print ("No Results")

Jurong East MRT Station,Lat: 1.33329506563598 Long: 103.742153884191
Bukit Batok MRT Station,Lat: 1.34903331201636 Long: 103.749566478309
Bukit Gombak MRT Station,Lat: 1.35861159094192 Long: 103.751790910733
Choa Chu Kang MRT Station,Lat: 1.38536316540225 Long: 103.744370779756
Yew Tee MRT Station,Lat: 1.39729771118639 Long: 103.74735767326
Kranji MRT Station,Lat: 1.4250690629411 Long: 103.762099449562
Marsiling MRT Station,Lat: 1.43262603485889 Long: 103.774051197031
Woodlands MRT Station,Lat: 1.43681962961519 Long: 103.786066799253
Admiralty MRT Station,Lat: 1.44058856161847 Long: 103.800990519771
Sembawang MRT Station,Lat: 1.44902668521092 Long: 103.820139486468
Canberra MRT Station,Lat: 1.44307664075699 Long: 103.829702590959
Yishun MRT Station,Lat: 1.42952522867762 Long: 103.834995141847
Khatib MRT Station,Lat: 1.41758906054002 Long: 103.833061043125
Yio Chu Kang MRT Station,Lat: 1.38149882492597 Long: 103.845170704861
Ang Mo Kio MRT Station,Lat: 1.36993284962264 Long: 103.8495580

In [16]:
mrt_location = pd.DataFrame({
    'MRT': list_of_mrt,
    'latitude': mrt_lat,
    'longitude': mrt_long
})

# Getting coordinates of each HDB(Housing and Developing Board) Resale flat
in order to conduct the distance from MRT stations as well as

*   So, that we can take out the distance of flats from MRT stations (Mass Rapid Transit System).
*   And, also from CBD (Central Business District).


In [17]:
df['address'] = df['block'] + " " + df['street_name']
df

C:\Users\HP\AppData\Local\Temp\ipykernel_8696\2900962711.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['address'] = df['block'] + " " + df['street_name']


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,address
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,255000.0,70,174 ANG MO KIO AVE 4
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,275000.0,65,541 ANG MO KIO AVE 10
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,285000.0,64,163 ANG MO KIO AVE 4
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,290000.0,63,446 ANG MO KIO AVE 10
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,290000.0,64,557 ANG MO KIO AVE 10
...,...,...,...,...,...,...,...,...,...,...,...,...
182350,2024-06,YISHUN,EXECUTIVE,328,YISHUN RING RD,04 TO 06,142.0,Apartment,1988,850000.0,63 years 01 month,328 YISHUN RING RD
182351,2024-06,YISHUN,EXECUTIVE,723,YISHUN ST 71,10 TO 12,146.0,Maisonette,1986,790000.0,61 years,723 YISHUN ST 71
182352,2024-06,YISHUN,EXECUTIVE,826,YISHUN ST 81,10 TO 12,146.0,Maisonette,1988,1000000.0,62 years 08 months,826 YISHUN ST 81
182353,2024-06,YISHUN,EXECUTIVE,826,YISHUN ST 81,04 TO 06,146.0,Maisonette,1988,1000000.0,62 years 08 months,826 YISHUN ST 81


In [18]:
address_list = df['address'].unique()

In [21]:
latitude = []
longitude = []
blk_no = []
road_name = []
postal_code = []
address = []
count = 0

for row in range(len(address_list)):
    query_address = address_list[row]
    query_string='https://www.onemap.gov.sg/api/common/elastic/search?searchVal='+str(query_address)+'&returnGeom=Y&getAddrDetails=Y'
    resp = requests.get(query_string)


    data_geo_location=json.loads(resp.content)
    if data_geo_location['found'] != 0:
        latitude.append(data_geo_location['results'][0]['LATITUDE'])
        longitude.append(data_geo_location['results'][0]['LONGITUDE'])
        blk_no.append(data_geo_location['results'][0]['BLK_NO'])
        road_name.append(data_geo_location['results'][0]['ROAD_NAME'])
        postal_code.append(data_geo_location['results'][0]['POSTAL'])
        address.append(query_address)
        print (str(query_address) + " ,Lat: " + data_geo_location['results'][0]['LATITUDE'] + " Long: " + data_geo_location['results'][0]['LONGITUDE'])
    else:
        print ("No Results")

174 ANG MO KIO AVE 4 ,Lat: 1.37509746867904 Long: 103.83761896123
541 ANG MO KIO AVE 10 ,Lat: 1.37392239168826 Long: 103.855621371068
163 ANG MO KIO AVE 4 ,Lat: 1.37354853919927 Long: 103.838176471398
446 ANG MO KIO AVE 10 ,Lat: 1.36776095130953 Long: 103.855357145908
557 ANG MO KIO AVE 10 ,Lat: 1.3716257020332 Long: 103.857736107527
603 ANG MO KIO AVE 5 ,Lat: 1.38020079047279 Long: 103.83575571651
709 ANG MO KIO AVE 8 ,Lat: 1.37113720765377 Long: 103.847662320064
333 ANG MO KIO AVE 1 ,Lat: 1.3613425564061 Long: 103.851698621454
109 ANG MO KIO AVE 4 ,Lat: 1.3700965375834 Long: 103.837687766047
564 ANG MO KIO AVE 3 ,Lat: 1.36984837555524 Long: 103.859404131956
218 ANG MO KIO AVE 1 ,Lat: 1.36511908595886 Long: 103.841742478489
556 ANG MO KIO AVE 10 ,Lat: 1.37203236308683 Long: 103.857625375797
156 ANG MO KIO AVE 4 ,Lat: 1.37549519574964 Long: 103.839947470774
471 ANG MO KIO AVE 10 ,Lat: 1.36346600647245 Long: 103.856702918462
434 ANG MO KIO AVE 10 ,Lat: 1.3678915069194 Long: 103.85345502

ReadTimeout: HTTPSConnectionPool(host='www.onemap.gov.sg', port=443): Read timed out. (read timeout=None)

In [22]:
df_coordinates = pd.DataFrame({
    'latitude': latitude,
    'longitude': longitude,
    'blk_no': blk_no,
    'road_name': road_name,
    'postal_code': postal_code,
    'address': address
})
len(df_coordinates)

2325

In [ ]:
list_of_lat = df_coordinates['latitude']
list_of_long = df_coordinates['longitude']
mrt_lat = mrt_location['latitude']
mrt_long = mrt_location['longitude']

In [ ]:
list_of_coordinates = []
list_of_mrt_coordinates = []

for lat, long in zip(list_of_lat, list_of_long):
    list_of_coordinates.append((lat,long))
for lat, long in zip(mrt_lat, mrt_long):
    list_of_mrt_coordinates.append((lat, long))

# Getting the diatnce with the help of OpenMap API

In [ ]:
# Distance to nearest MRT
from geopy.distance import geodesic

list_of_dist_mrt = []
min_dist_mrt = []

for origin in list_of_coordinates:
    for destination in range(0, len(list_of_mrt_coordinates)):
        list_of_dist_mrt.append(geodesic(origin,list_of_mrt_coordinates[destination]).meters)
    shortest = (min(list_of_dist_mrt))
    min_dist_mrt.append(shortest)
    list_of_dist_mrt.clear()

In [ ]:
# Distance from CDB
cbd_dist = []

for origin in list_of_coordinates:
    cbd_dist.append(geodesic(origin,(1.2830, 103.8513)).meters) #CBD coordinates

In [ ]:
# Put MRT and CBD distance together
df_coordinates['cbd_dist'] = cbd_dist
df_coordinates['min_dist_mrt'] = min_dist_mrt

In [ ]:
df_coordinates

In [ ]:
df_coordinates.to_csv('C:/Users/HP/Desktop/Data Scientist/project/singapore resale flat price prediction/New data/df_coordinates1.csv',index=False)

In [ ]:
df_coordinates = pd.read_csv('C:/Users/HP/Desktop/Data Scientist/project/singapore resale flat price prediction/New data/df_coordinates1.csv')

In [ ]:
df_new = df_coordinates.merge(df, on="address", how='outer')
df_new

In [ ]:
df_new['resale_price'] = df_new['resale_price'].astype('float')
df_new['floor_area_sqm'] = df_new['floor_area_sqm'].astype('float')
df_new['lease_commence_date'] = df_new['lease_commence_date'].astype('int64')
df_new['lease_remain_years'] = 99 - (2023 - df_new['lease_commence_date'])

df_new.dropna(inplace=True)

df_new

In [ ]:
df_new.to_csv('C:/Users/HP/Desktop/Data Scientist/project/singapore resale flat price prediction/New data/combined1.csv', index = False)